In [ ]:
import csv

from math import log
from pprint import pprint
from collections import defaultdict

from ipywidgets import HTML
from ipywidgets.embed import embed_minimal_html
from ipyleaflet import Map, Marker, Popup, basemaps, basemap_to_tiles, CircleMarker

In [ ]:
smugglers = list(csv.DictReader(open("data/dunkerque_smugglers_shipmentvalues.csv", 'r')))

In [ ]:
pprint(list(smugglers[0].keys()))

In [ ]:
homeports_shipments = defaultdict(int)
homeports_origin = {}
for boat in smugglers:
    if int(boat["shipment_price"]) > 0 :
        homeports_shipments[boat["homeport_standardized_fr"].replace("\xa0", "")] += int(boat["shipment_price"])
        homeports_origin[boat["homeport_standardized_fr"].replace("\xa0", "")] = (boat["homeport_lat"].replace(",", "."), boat["homeport_long"].replace(",", "."))

homeports_origin.pop('pas identifié', None)    
homeports_shipments.pop('pas identifié', None)  
homeports_origin.pop('', None)    
homeports_shipments.pop('', None)  
pprint(homeports_shipments)

Champs de *smugglers* avec coordonnées géo :

- `homeport_lat`
- `homeport_long`
- `pointcall_lat`
- `pointcall_long`
- `homeport_geom4326`
- `pointcall_geom4326`

Ex :

```python
marker = Marker(
    location=(flow["homeport_latitude"], flow["homeport_longitude"])
)
```

In [ ]:
center = (51, 1) # France
smugglers_map = Map(center=center,zoom=7)

for port in homeports_shipments:
    
    circle_marker = CircleMarker()
    circle_marker.location = (homeports_origin[port][0], homeports_origin[port][1])
    circle_marker.radius = round(log(homeports_shipments[port] / 1000, 2)+5)
    circle_marker.color = "black"
    circle_marker.fill_color = "red"
    
    popup = HTML()
    popup.value = str(port)
    popup = Popup(
        location=(homeports_origin[port][0], homeports_origin[port][1]),
        child=popup
    )
    circle_marker.popup = popup

    smugglers_map.add_layer(circle_marker)
        
display(smugglers_map)
embed_minimal_html('viz_thesmogglepart/homeports_smuggled_shipments_values.html', views=[smugglers_map])